In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
from environments.base_env import HomeostaticEnvironment
from environments.anticipatory_env import AnticipatoryEnvironment
from models.qlearning import QLearning

# Test base_env 

In [94]:
import torch
# Définir les paramètres
initial_state = [90, 37]  # Glucose = 90, Température = 37
setpoints = [100, 37]  # Setpoints pour le glucose et la température
weights = [1, 2]  # Importance relative : température plus critique
exponents = [2, 2]  # Non-linéarité pour les calculs de drive
effects = {
    "eat": [10, 0],  # Manger augmente le glucose de 10
    "cool_down": [0, -2]  # Se refroidir réduit la température de 2
}

# Instancier l'environnement
env = HomeostaticEnvironment(H=initial_state, setpoints=setpoints, weights=weights, exponents=exponents, effects=effects)

# Test de l'environnement
print("État initial :", env.state)

# Effectuer une action ("eat")
state, reward,_,_ = env.step("eat")
print("État après manger :", state)
print("Récompense :", reward)

# Effectuer une autre action ("cool_down")
state, reward,_,_ = env.step("cool_down")
print("État après refroidissement :", state)
print("Récompense :", reward)


État initial : tensor([90., 37.])
État après manger : tensor([100.,  37.])
Récompense : tensor(10.)
État après refroidissement : tensor([100.,  35.])
Récompense : tensor(-2.8284)


# Test Experience Anticipation avec QLearning 

In [106]:
# Paramètres
state_size = 1  # la température
action_size = 2  # Actions : [0 = aucune réponse, 1 = réponse anticipatoire]
alpha = 0.1
gamma = 0.99
epsilon = 0.1

# Instancier le modèle Q-learning
agent = QLearning(state_size=state_size, action_size=action_size, alpha=alpha, gamma=gamma, epsilon=epsilon)


In [108]:
# Définition des paramètres de l'environnement
initial_state = torch.tensor([37.0])  # Température corporelle initiale
setpoints = torch.tensor([37.0])  # Température optimale
weights = torch.tensor([1.0])  # Importance de la température
exponents = [2, 2]  # Non-linéarité
effects = {
    "anticipate": torch.tensor([0.5]),  # Augmentation légère de la température
    "injection": torch.tensor([-1.5]),  # Diminution importante de la température
}
signal_timesteps = [10, 20, 30]  # Moments où un signal est donné avant l'injection

# Instanciation de l'environnement
env = AnticipatoryEnvironment(
    H=initial_state,
    setpoints=setpoints,
    weights=weights,
    exponents=exponents,
    effects=effects,
    signal_timesteps=signal_timesteps,
)
